In [1]:
'''This script is adapted from the blog post

"Building powerful image classification models using very little data"

from blog.keras.io.

So that we have 1000 training examples for each class, and 400 validation examples for each class.

In summary, this is our directory structure:

```

data/

    train/

        livebox_play/

            livebox_1.jpg

            livebox_2.jpg

            ...

        other/

            livebox_1.jpg

            livebox_2.jpg

            ...

    validation/

        livebox_play/

            livebox_1.jpg

            livebox_2.jpg

            ...

        other/

            other_1.jpg

            other_2.jpg

            ...

```

'''


import keras

import tensorflow

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K

print('keras version:' + keras.__version__)
print('tensorflow version:' + tensorflow.__version__)






# dimensions of our images.

img_width, img_height = 150, 150

train_data_dir = '/home/comedelobelle/Documents/Environment/data/train'

validation_data_dir = '/home/comedelobelle/Documents/Environment/data/validation'

nb_train_samples = 8000 #2700

nb_validation_samples = 2400#600

epochs = 80

batch_size = 40



if K.image_data_format() == 'channels_first':

    input_shape = (3, img_width, img_height)

else:
    
    input_shape = (img_width, img_height, 3)


model = Sequential()


model.add(Conv2D(32, (3, 3), input_shape=input_shape))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Conv2D(64, (3, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Flatten())

model.add(Dense(64))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(8))

model.add(Activation('softmax')) #use sigmoid when binary and softmax when categorical


model.compile(loss='categorical_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])



# this is the augmentation configuration we will use for training

train_datagen = ImageDataGenerator(

    rescale=1. / 255,

    shear_range=0.2,

    zoom_range=0.2,

    horizontal_flip=True)



# this is the augmentation configuration we will use for testing:

# only rescaling

test_datagen = ImageDataGenerator(rescale=1. / 255)



train_generator = train_datagen.flow_from_directory(

    train_data_dir,

    target_size=(img_width, img_height),

    batch_size=batch_size,

    class_mode='categorical')



validation_generator = test_datagen.flow_from_directory(

    validation_data_dir,

    target_size=(img_width, img_height),

    batch_size=batch_size,

    class_mode='categorical')


model.fit_generator(

    train_generator,

    steps_per_epoch=nb_train_samples // batch_size,

    epochs=epochs,

    validation_data=validation_generator,

    validation_steps=nb_validation_samples // batch_size)


model.save('discriminate_model.h5')
model.save_weights('discriminate_weights.h5')

Using TensorFlow backend.


keras version:2.0.5
tensorflow version:1.1.0
Found 3221 images belonging to 8 classes.
Found 446 images belonging to 8 classes.
Epoch 1/80
200/200 [==============================] - 534s - loss: 1.4966 - acc: 0.4584 - val_loss: 0.8381 - val_acc: 0.7601
Epoch 2/80
200/200 [==============================] - 378s - loss: 0.9810 - acc: 0.6672 - val_loss: 0.6020 - val_acc: 0.8466
Epoch 3/80
200/200 [==============================] - 379s - loss: 0.8214 - acc: 0.7216 - val_loss: 0.5014 - val_acc: 0.8686
Epoch 4/80
200/200 [==============================] - 375s - loss: 0.6818 - acc: 0.7726 - val_loss: 0.4003 - val_acc: 0.8897
Epoch 5/80
200/200 [==============================] - 375s - loss: 0.6187 - acc: 0.7923 - val_loss: 0.5620 - val_acc: 0.8332
Epoch 6/80
200/200 [==============================] - 383s - loss: 0.5647 - acc: 0.8079 - val_loss: 0.3463 - val_acc: 0.8937
Epoch 7/80
200/200 [==============================] - 380s - loss: 0.5538 - acc: 0.8157 - val_loss: 0.3808 - val_acc: 0.89

200/200 [==============================] - 386s - loss: 0.8169 - acc: 0.7735 - val_loss: 0.7268 - val_acc: 0.7197
Epoch 66/80
200/200 [==============================] - 385s - loss: 0.8035 - acc: 0.7808 - val_loss: 0.5305 - val_acc: 0.8830
Epoch 67/80
200/200 [==============================] - 386s - loss: 0.7215 - acc: 0.7932 - val_loss: 0.4544 - val_acc: 0.8767
Epoch 68/80
200/200 [==============================] - 3460s - loss: 0.7570 - acc: 0.7786 - val_loss: 0.4562 - val_acc: 0.8574
Epoch 69/80
200/200 [==============================] - 397s - loss: 0.7068 - acc: 0.7981 - val_loss: 0.5179 - val_acc: 0.8278
Epoch 70/80
200/200 [==============================] - 396s - loss: 0.7991 - acc: 0.7812 - val_loss: 0.6965 - val_acc: 0.7996
Epoch 71/80
200/200 [==============================] - 391s - loss: 0.7609 - acc: 0.7716 - val_loss: 0.4839 - val_acc: 0.8534
Epoch 72/80
200/200 [==============================] - 389s - loss: 0.7479 - acc: 0.7967 - val_loss: 0.6749 - val_acc: 0.8839
Epo

In [2]:
#run the model for Livebox

import keras

import tensorflow

import cv2

import numpy as np

from PIL import Image

import glob

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential

from keras.models import load_model

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K

from keras.preprocessing.image import img_to_array, load_img

from keras.backend import tf as ktf

print('keras version:' + keras.__version__)

print('tensorflow version:' + tensorflow.__version__)

model = load_model('discriminate_model.h5') # loading model and weights for the prediction
model.load_weights('discriminate_weights.h5')
model.compile(loss='categorical_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])

#Confirm on one single image from any validation set
img = load_img('/home/comedelobelle/Documents/Environment/data/fakecheck.jpg') # this is a PIL image

x = img_to_array(img)  
imgresized = cv2.resize(x,(150,150)) # this is a Numpy array with shape (3, 150, 150)

imgresized = imgresized[np.newaxis,...] # dimension added to fit input size
predo = model.predict_proba(imgresized,1)
print(predo)

#Confirm on Livebox 2 validation set
print('\n\n\n\nLivebox 2 set test')

imageFolderPath2 = '/home/comedelobelle/Documents/Environment/data/validation/livebox_2'
imagePath2 = glob.glob(imageFolderPath2 + '/*.jpg')
'''im_array2 = np.array( [cv2.resize(np.array(Image.open(imagePath2[i])), (150,150)) for i in range(len(imagePath2))] )
predo2 = model.predict_proba(im_array2)
print(predo2)'''
live2 = 0
live4 = 0
livePlay = 0

for k in range(len(imagePath2)):
    try: # print the category using the model.predict output
        im_array2 = np.array([cv2.resize(np.array(Image.open(imagePath2[k])), (150,150))])
        predo2 = model.predict(im_array2)
        print(predo2)
        itemindex = np.flatnonzero(predo2==1)
        if itemindex == 2:
            live2 += 1
            print('C\'est une Livebox 2')
        elif itemindex == 3:
            live4 += 1
            print('C\'est une Livebox 4')
        elif itemindex == 4:
            livePlay += 1
            print('C\'est une Livebox Play')
    except IndexError: # except the error occurring when no solid prediction was made
        print('Aucune prédiction certaine\nDonnées d\'entrainement du modèle insuffisantes')

print(live2)
print(live4)
print(livePlay)

taux2 = ((live4 + livePlay)/(live2+live4+livePlay))*100
taux2 = str(taux2)
print('\nTaux d\'erreur = ' +taux2+ ' %')

#Confirm on Livebox 4 validation set
print('\n\n\n\nLivebox 4 set test')

imageFolderPath4 = '/home/comedelobelle/Documents/Environment/data/validation/livebox_4'
imagePath4 = glob.glob(imageFolderPath4 + '/*.jpg')
live2 = 0
live4 = 0
livePlay = 0
count=0

for k in range(len(imagePath4)):
    count += 1
    try: # print the category using the model.predict output
        im_array4 = np.array([cv2.resize(np.array(Image.open(imagePath4[k])), (150,150))])
        predo4 = model.predict(im_array4)
        print(predo4)
        itemindex = np.flatnonzero(predo4==1)
        if itemindex == 2:
            int live2 += 1
            print('C\'est une Livebox 2')
        elif itemindex == 3:
            live4 += 1
            print('C\'est une Livebox 4')
        elif itemindex == 4:
            livePlay += 1
            print('C\'est une Livebox Play')
    except IndexError: # except the error occurring when no solid prediction was made
        print('Aucune prédiction certaine\nDonnées d\'entrainement du modèle insuffisantes')

print(live2)
print(live4)
print(livePlay)       

taux4 = ((live2 + livePlay)/count)*100
taux4 = str(taux4)
print('\nTaux d\'erreur = ' +taux4+ ' %')

#Confirm on Livebox Play validation set
print('\n\n\n\nLivebox Play set test')

imageFolderPathPlay = '/home/comedelobelle/Documents/Environment/data/validation/livebox_play'
imagePathPlay = glob.glob(imageFolderPathPlay + '/*.jpg')
live2 = 0
live4 = 0
livePlay = 0
count=0
for k in range(len(imagePathPlay)):
    count += 1
    try: # print the category using the model.predict output
        im_arrayPlay = np.array([cv2.resize(np.array(Image.open(imagePathPlay[k])), (150,150))])
        predoPlay = model.predict(im_arrayPlay)
        print(predoPlay)
        itemindex = np.flatnonzero(predoPlay==1)
        if itemindex == 2:
            live2 += 1
            print('C\'est une Livebox 2')
        elif itemindex == 3:
            live4 += 1
            print('C\'est une Livebox 4')
        elif itemindex == 4:
            livePlay += 1
            print('C\'est une Livebox Play')
    except IndexError: # except the error occurring when no solid prediction was made
        print('Aucune prédiction certaine\nDonnées d\'entrainement du modèle insuffisantes')

print(live2)
print(live4)
print(livePlay)

tauxPlay = ((live2+live4)/count)*100
print tauxPlay
tauxPlay = str(tauxPlay)
print('\nTaux d\'erreur = ' +tauxPlay+ ' %')

keras version:2.0.5
tensorflow version:1.1.0
1/1 [==============================] - 0s
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]




Livebox 2 set test
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.16957743  0.15669265  0.09717064  0.0731681   0.03393853  0.13745792
   0.19020861  0.14178605]]
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.  0.  1.  0.  0.  0.  0.  0.]]
C'est une Livebox 2
[[ 0.16957743  0.15669265  0.09717064  0.0731681   0.03393853  0.13745792
   0.1

[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.16957743  0.15669265  0.09717064  0.0731681   0.03393853  0.13745792
   0.19020861  0.14178605]]
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  0.  0.  0.  1.]]
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.16957743  0.15669265  0.09717064  0.0731681   0.03393853  0.13745792
   0.19020861  0.14178605]]
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebox Play
[[ 0.  0.  0.  0.  1.  0.  0.  0.]]
C'est une Livebo

In [ ]:
import coremltools

coreml_model = coremltools.converters.keras.convert('fool-enhanced_model.h5',
                                                    
                                                    input_names = 'image',
                                                    
                                                    image_input_names = 'image',
                                                    
                                                    class_labels = ['Livebox 2', 'Livebox 4', 'Livebox Play'])
coreml_model.save("livebox_recV3.mlmodel")

coreml_model

In [ ]:
#Android Compiler model + weights

import keras

import tensorflow

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K

print('keras version:' + keras.__version__)
print('tensorflow version:' + tensorflow.__version__)

from keras.layers import InputLayer

input = tf.placeholder(tf.float32, name='input')



model = Sequential()

model.add(InputLayer(input_tensor=input))




model.add(Conv2D(32, (3, 3), input_shape=input_shape))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Conv2D(64, (3, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Flatten())

model.add(Dense(64))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(3))

model.add(Activation('softmax')) #use sigmoid when binary and softmax when categorical



model.compile(loss='categorical_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])

In [ ]:
#Android adaptation from the original program

'''This script is adapted from the blog post

"Building powerful image classification models using very little data"

from blog.keras.io.

So that we have 1000 training examples for each class, and 400 validation examples for each class.

In summary, this is our directory structure:

```

data/

    train/

        livebox_play/

            livebox_1.jpg

            livebox_2.jpg

            ...

        other/

            livebox_1.jpg

            livebox_2.jpg

            ...

    validation/

        livebox_play/

            livebox_1.jpg

            livebox_2.jpg

            ...

        other/

            other_1.jpg

            other_2.jpg

            ...

```

'''


import keras

import tensorflow
sess=tensorflow.Session()

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K
K.set_session(sess)

from keras.layers import Input
from keras.layers import InputLayer

print('keras version:' + keras.__version__)
print('tensorflow version:' + tensorflow.__version__)






# dimensions of our images.

img_width, img_height = 150, 150



train_data_dir = '/home/comedelobelle/Documents/Environment/data/train'

validation_data_dir = '/home/comedelobelle/Documents/Environment/data/validation'

nb_train_samples = 120 #2700

nb_validation_samples = 24 #600

epochs = 5 #50

batch_size = 15 #30



if K.image_data_format() == 'channels_first':

    input_shape = (3, img_width, img_height)
    input_shapep = (0, 3, img_width, img_height)

else:
    
    input_shape = (img_width, img_height, 3)
    input_shapep = (0, img_width, img_height, 3)
    

#inputp = tensorflow.placeholder(tensorflow.float32, shape=input_shapep) #android test



model = Sequential()

#model.add(InputLayer(input_tensor=inputp, input_shape=input_shape)) #android test




model.add(Conv2D(32, (3, 3), input_shape=input_shape))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Flatten())

model.add(Dense(64))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(3))

model.add(Activation('softmax')) #use sigmoid when binary and softmax when categorical


model.compile(loss='categorical_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])



# this is the augmentation configuration we will use for training

train_datagen = ImageDataGenerator(

    rescale=1. / 255,

    shear_range=0.2,

    zoom_range=0.2,

    horizontal_flip=True)



# this is the augmentation configuration we will use for testing:

# only rescaling

test_datagen = ImageDataGenerator(rescale=1. / 255)



train_generator = train_datagen.flow_from_directory(

    train_data_dir,

    target_size=(img_width, img_height),

    batch_size=batch_size,

    class_mode='categorical')



validation_generator = test_datagen.flow_from_directory(

    validation_data_dir,

    target_size=(img_width, img_height),

    batch_size=batch_size,

    class_mode='categorical')


model.fit_generator(

    train_generator,

    steps_per_epoch=nb_train_samples // batch_size,

    epochs=epochs,

    validation_data=validation_generator,

    validation_steps=nb_validation_samples // batch_size)




# Export to Tensorflow
from tensorflow.python.saved_model import builder, tag_constants, signature_def_utils, loader, utils

sess = K.get_session()
tensorflow.initialize_all_variables().run(session=sess)
saver = builder.SavedModelBuilder('/home/comedelobelle/Documents/Environment/TFmodelAnd2/')
signature = signature_def_utils.build_signature_def(inputs={'conv2d': utils.build_tensor_info(model.input)},
                                              outputs={'outputs': utils.build_tensor_info(model.output)},
                                              method_name='tensorflow/serving/predict')
saver.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING],
                                             signature_def_map={'predict': signature})
saver.save()

model.save('androidtest_model.h5')
model.save_weights('androidtest_weights.h5')

In [ ]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # Livebox images are 150x150 pixels, and have three color channel
  input_layer = tf.reshape(features, [-1, 150, 150, 3], name='inputlayer')

  # Convolutional Layer #1
  # Computes 32 features using a 3x3 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 150, 150, 3]
  # Output Tensor Shape: [batch_size, 150, 150, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 150, 150, 32]
  # Output Tensor Shape: [batch_size, 75, 75, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
  # Convolutional Layer #2
  # Computes 32 features using a 3x3 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 150, 150, 3]
  # Output Tensor Shape: [batch_size, 150, 150, 32]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 150, 150, 32]
  # Output Tensor Shape: [batch_size, 75, 75, 32]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Convolutional Layer #3
  # Computes 64 features using a 3x3 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 75, 75, 32]
  # Output Tensor Shape: [batch_size, 75, 75, 64]
  conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #3
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 75, 75, 64]
  # Output Tensor Shape: [batch_size, 37, 37, 64]
  pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 37, 37, 64]
  # Output Tensor Shape: [batch_size, 37 * 37 * 64]
  pool3_flat = tf.reshape(pool3, [-1, 37 * 37 * 64])

  # Dense Layer
  # Densely connected layer with 64 neurons
  # Input Tensor Shape: [batch_size, 37 * 37 * 64]
  # Output Tensor Shape: [batch_size, 64]
  dense = tf.layers.dense(inputs=pool3_flat, units=64, activation=tf.nn.relu)

  # Add dropout operation; 0.5 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.5, training=mode == learn.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 64]
  # Output Tensor Shape: [batch_size, 3]
  logits = tf.layers.dense(inputs=dropout, units=3)

  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="SGD")

  # Generate Predictions
  predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

#Load custom train dataset for Livebox



def main(unused_argv):
  # Load training and eval data
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  mnist_classifier.fit(
      x=train_data,
      y=train_labels,
      batch_size=100,
      steps=20000,
      monitors=[logging_hook])

  # Configure the accuracy metric for evaluation
  metrics = {
      "accuracy":
          learn.MetricSpec(
              metric_fn=tf.metrics.accuracy, prediction_key="classes"),
  }

  # Evaluate the model and print results
  eval_results = mnist_classifier.evaluate(
      x=eval_data, y=eval_labels, metrics=metrics)
  print(eval_results)


if __name__ == "__main__":
    tf.app.run()

In [ ]:
import glob
import tensorflow as tf
from os import listdir
from os.path import isfile, join

livebox2train = [f for f in listdir('./data/train/livebox_2') if isfile(join('./data/train/livebox_2', f))]
livebox2train_queue = tf.train.string_input_producer(livebox2train)

reader = tf.WholeFileReader()
livebox2train, content2 = reader.read(livebox2train_queue)
image2 = tf.image.decode_jpeg(content2, channels=3)
image2 = tf.cast(image2, tf.float32)
resized_image2 = tf.image.resize_images(image2, [150, 150])

image2_batch = tf.train.batch([resized_image2, 'livebox_2'], batch_size=8)

livebox4train = [f for f in listdir('./data/train/livebox_4') if isfile(join('./data/train/livebox_4', f))]
livebox4train_queue = tf.train.string_input_producer(livebox4train)

livebox4train, content4 = reader.read(livebox4train_queue)
image4 = tf.image.decode_jpeg(content4, channels=3)
image4 = tf.cast(image4, tf.float32)
resized_image4 = tf.image.resize_images(image4, [150, 150])

image4_batch = tf.train.batch([resized_image4, 'livebox_4'], batch_size=8)

liveboxPlaytrain = [f for f in listdir('./data/train/livebox_play') if isfile(join('./data/train/livebox_play', f))]
liveboxPlaytrain_queue = tf.train.string_input_producer(liveboxPlaytrain)

liveboxPlaytrain, contentPlay = reader.read(liveboxPlaytrain_queue)
imagePlay = tf.image.decode_jpeg(contentPlay, channels=3)
imagePlay = tf.cast(imagePlay, tf.float32)
resized_imagePlay = tf.image.resize_images(imagePlay, [150, 150])

imagePlay_batch = tf.train.batch([resized_imagePlay, 'livebox_play'], batch_size=8)
print(imagePlay_batch)